In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from functools import partial
import parallel
import matplotlib.pyplot as plt
from dbn.models import SupervisedDBNRegression
from elm.elm import Model as elm
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.base import clone
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import ParameterGrid
import sklearn.metrics as metrics
from sklearn import preprocessing
from statsmodels.tsa.stattools import adfuller
from multiprocessing import Pool
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
import itertools

In [2]:
file_names = ["amz", "APPLE", "electricity", "goldman", "msft", "pollutions", "star", "sunspot", "vehicle", "wine"]
file_prefix = "dataset/"
file_suffix = ".txt"

ensemble_svr = {'windows': True, 'lags': 20, 'normalize_before_metrics': True, 'differencing_order': 'ADF', 'level': '5%', 'p_level': 0.05}
ens_size = 100

execution="oracle" # dsnaw / dsla / oracle

plot_curves = False
train_proportion = 1/2
val_proportion = 1/4
max_k = 20
max_n = min(20, ens_size)

In [3]:
#read files
def read_file(path):
    array_ts = np.loadtxt(path)
    df = pd.DataFrame(data = array_ts, columns = ['y'])
    return df

In [4]:
#define split points

def define_split_points(df, train, val):
    val_index = int(df.shape[0]*train)
    test_index = int(df.shape[0]*(train+val))

    # fig, ax=plt.subplots(figsize=(9, 4))
    # df['y'].loc[:val_index-1].plot(ax=ax, label = 'train')
    # df['y'].loc[val_index:test_index-1].plot(ax=ax, label = 'validation')
    # df['y'].loc[test_index:].plot(ax=ax, label = 'test')
    # ax.legend()
    
    return val_index, test_index


In [5]:
#difference time series

def check_ADF(ts, lvl, p_lvl):
    result = adfuller(ts)
    return result[0] < result[4][lvl] and result[1] < p_lvl

def difference(df, differencing_order, level, p_level):

    df_differenced = None
    if differencing_order:    
        df_differenced = df.copy()

        if differencing_order == 'ADF':
            stationary = check_ADF(df_differenced['y'].values, level, p_level)
            d = 0
            while not stationary:
                df_differenced = df_differenced.diff().dropna()
                d += 1
                stationary = check_ADF(df_differenced['y'].values, level, p_level)
            if d == 0:
                df_differenced = None
            differencing_order = d
        else:
            i = 0
            while i < differencing_order:
                df_differenced = df_differenced.diff().dropna()
                i+=1
                
    print("d: ", differencing_order)
    # if differencing_order:
    #     fig, ax=plt.subplots(figsize=(9, 4))
    #     df_differenced.y.plot(ax = ax)
        
    return df_differenced, differencing_order


In [6]:
#normalize time series

def normalize(df, val_index, d):
    min_max_scaler = preprocessing.MinMaxScaler()
    min_max_scaler.fit(df['y'].loc[:val_index-1].values.reshape(-1,1))
    df_normalized = pd.DataFrame({'y': np.append(np.zeros(d),min_max_scaler.transform(df['y'].values.reshape(-1, 1)).flatten())}).iloc[d:]
    
    return df_normalized, min_max_scaler


In [7]:
#create windows

def create_windows(df, lags):
    df_windowed = df.copy()
    df_windowed['x'] = df_windowed['y'].shift()
    for i in range(1, lags):
        df_windowed[f'x-{i}'] = df_windowed['x'].shift(i)
    df_windowed = df_windowed.dropna()
    return df_windowed

In [8]:
#SVR

def svr_model(lags):
    parameters = {'C':[1, 10, 100, 1000], 'gamma': [0.001, 0.01, 0.1, 1],
                      'kernel':["rbf"],
                      'epsilon': [0.1, 0.01, 0.001, 0.0001, 0.00001],
                 }

    model = SVR(max_iter = 10000)

    return model, parameters

In [9]:
def recursive_differencing_coefficients(order):
    if order <= 1:
        return np.array([1, -1])
    else:
        prev_coefficients = recursive_differencing_coefficients(order-1)
        coefficients = np.zeros(order+1)
        coefficients[0] = prev_coefficients[0]
        coefficients[-1] = -prev_coefficients[-1]
        for i in range(1, order):
            coefficients[i] = prev_coefficients[i] - prev_coefficients[i-1]
        return coefficients   

def inverse_difference(differencing_order, df, df_diff, test_index, y_pred):
    df_pred = df.copy()
    
    for i in range(1, differencing_order + 1):
        df_pred[f'y-{i}'] = df_pred['y'].shift(i)
    
    df_pred['y_diff'] = df_diff['y']
    df_pred['y_diff'][test_index:] = y_pred
    
    df_pred.fillna(0, inplace=True)
    
    coefficients = recursive_differencing_coefficients(differencing_order)
    df_pred['y_pred'] = df_pred['y_diff']
    for i in range(1, differencing_order + 1):
        df_pred['y_pred'] = df_pred['y_pred'] - coefficients[i]*df_pred[f'y-{i}']
        
    y_pred = df_pred['y_pred'][test_index:].values    
    y_test = df_pred['y'][test_index:].values
    
    return y_pred, y_test


In [10]:
#plot results
def plot_results(y_pred, y_test):
    fig, ax=plt.subplots(figsize=(9, 4))
    plt.plot(y_pred, label = 'predicted')
    plt.plot(y_test, label = 'real')
    ax.legend()
    plt.show()

In [11]:
def oracle(preds, real):
    best_pred = np.zeros(real.size)
    for i in range(real.size):
        candidates = np.array([p[i] for p in preds])
        idx = np.argmin(np.absolute(candidates - real[i]))
        best_pred[i] = candidates[idx]
    return best_pred

In [12]:
#calculate metrics

def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)

    posi_with_zeros = np.where(y_true == 0)[0]

    y_true = [n for k, n in enumerate(y_true) if k not in posi_with_zeros]
    y_pred = [n for k, n in enumerate(y_pred) if k not in posi_with_zeros]
    
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)

    posi_with_zeros = np.where(y_true == 0)[0]

    y_true = [n for k, n in enumerate(y_true) if k not in posi_with_zeros]
    y_pred = [n for k, n in enumerate(y_pred) if k not in posi_with_zeros]
    
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)

    posi_with_zeros = np.where((np.abs(y_true)+np.abs(y_pred))/2 == 0 )[0]

    y_true = [n for k, n in enumerate(y_true) if k not in posi_with_zeros]
    y_pred = [n for k, n in enumerate(y_pred) if k not in posi_with_zeros]
    
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return np.mean(np.abs(y_true - y_pred) / ((np.abs(y_true)+np.abs(y_pred))/2)) * 100


def mean_absolute_error(y_true, y_pred):
    
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)

    return np.mean(np.abs(y_true - y_pred))

def average_relative_variance(y_true, y_pred):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    mean = np.mean(y_true)

    error_sup = np.square(np.subtract(y_true, y_pred)).sum()
    error_inf = np.square(np.subtract(y_pred, mean)).sum()

    return error_sup / error_inf

def calculate_metrics(y_pred, y_test, normalize_before_metrics, scaler, df):
    if normalize_before_metrics:
        scaler.fit(df['y'].values.reshape(-1,1))
        y_pred = scaler.transform(y_pred.reshape(-1, 1)).flatten()
        y_test = scaler.transform(y_test.reshape(-1, 1)).flatten()

    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    nrmse = rmse/(y_test.max()-y_test.min())
    mape = mean_absolute_percentage_error(y_test, y_pred)
    smape = symmetric_mean_absolute_percentage_error(y_test, y_pred)
    arv = average_relative_variance(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    res = {'MSE': mse, 'MAPE': mape, 'ARV': arv, 'MAE': mae, 'RMSE': rmse, 'NRMSE': nrmse, 'SMAPE': smape}
    
    # for key, value in res.items():
    #     print(f'{key}: {value}')
    
    return res

In [13]:
#gridsearch

def parallel_gridsearch_ds(df, pred_results, val_index, test_index, param_grid, method, euclidians = None):
    pool = Pool()
    list_params = list(ParameterGrid(param_grid))
    if method == "dsnaw":
        results = list(tqdm(pool.imap(partial(parallel.validate_ds_params, df=df, pred_results=pred_results, val_index=val_index, test_index=test_index, method = "dsnaw"), list_params), total=len(list_params)))
    elif method == "dsla":
        results = list(tqdm(pool.imap(partial(parallel.validate_ds_params, df=df, pred_results=pred_results, val_index=val_index, test_index=test_index, method = "dsla", euclidians=euclidians), list_params), total=len(list_params)))

    
    idx = np.argmin([r[1] for r in results])
    
    best_params = results[idx][0]
    best_rmse = results[idx][1]
    
    pool.close()
    pool.join()
    
    print('Best params: ', best_params)
    print('Best rmse: ', best_rmse)
    return best_params

In [14]:
def test_ds(params, df, pred_results, val_index, test_index, method, euclidians=None):
    k = params['k']
    n = params['n']
    comb = params['comb']    
    
    y_dsnaw = np.zeros(df['y'].size-test_index)
    for i in range(test_index, df['y'].size):
        rmse_roc = []
        if method=="dsla":
            real_roc = df['y'].loc[euclidians[i-val_index,:k]].values
        else:
            real_roc = df['y'].loc[i-k:i-1].values
        for j in range(0, len(pred_results)):      
            pred_roc = pred_results[j]['y'].loc[i-k:i-1].values
            rmse_roc.append((j, np.sqrt(metrics.mean_squared_error(pred_roc, real_roc))))
        sorted_rmse_roc = sorted(rmse_roc, key=lambda x: x[1])
        
        comb_values = np.zeros(n)
        for j in range(0, n):
            comb_values[j] = pred_results[sorted_rmse_roc[j][0]]['y'].loc[i]
        if comb == 'median':
            y_dsnaw[i-test_index] = np.median(comb_values)
        else:
            y_dsnaw[i-test_index] = np.average(comb_values)

    return y_dsnaw

In [15]:
#test model

def test_model(model, df, test_index, scaler):   
    y_test = df['y'].loc[test_index:].values
    x_test = df.drop(columns = ['y']).loc[test_index:].values
    y_pred = model.predict(x_test)
    y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

    return y_pred, y_test

In [16]:
def parallel_gridsearch(estimator, param_grid, x_train, y_train, x_val, y_val):
    pool = Pool()
    list_params = list(ParameterGrid(param_grid))
    results = list(tqdm(pool.imap(partial(parallel.validate_params, estimator=estimator, x_train=x_train, 
                               y_train=y_train, x_val=x_val, y_val=y_val), list_params), total=len(list_params)))
    idx = np.argmin([r[1] for r in results])
    best_params = results[idx][0]
    best_rmse = results[idx][1]
    
    pool.close()
    pool.join()
    print('Best params: ', best_params)
    print('Best rmse: ', best_rmse)
    return best_params

In [17]:
@ignore_warnings(category=ConvergenceWarning)
def train_svr(x_train, y_train, x_val, y_val):
    
    param_grid = {'C':[1, 10, 100, 1000], 'gamma': [0.001, 0.01, 0.1, 1],
                      'kernel':["rbf"],
                      'epsilon': [0.1, 0.01, 0.001, 0.0001, 0.00001],
                 }

    model = SVR(max_iter = 10000)
    
    best_params = parallel_gridsearch(model, param_grid, 
                                    x_train, y_train,
                                    x_val, y_val)

    model.set_params(**best_params)
    model.fit(x_train, y_train)

    
    # for k, g, e, c in hyper_param:
    #     modelo = SVR(kernel=k,gamma=g, epsilon=e, C=c, max_iter = 10000)
    #     modelo.fit(x_train, y_train)
    #     prev_v = modelo.predict(x_val)
    #     novo_mse  = metrics.mean_squared_error(y_val, prev_v)
    #     if novo_mse < melhor_mse:
    #         melhor_mse = novo_mse
    #         melhor_modelo = modelo

    return model, None

In [18]:
def reamostragem(serie, n):
    size = len(serie)
    #nova_particao = []
    ind_particao = []
    for i in range(n):
        ind_r = np.random.randint(size)
        ind_particao.append(ind_r)
        #nova_particao.append(serie[ind_r,:])
    
    return ind_particao

In [19]:
def bagging(qtd_modelos, X_train, Y_train, val_index):
    
    ens = []
    ensemble = {'models':[], 'indices': [] }   
    ind_particao = []
    
    # if len(y_train.shape) == 1:
    #     y_train =  y_train.reshape(len(y_train), 1)
    
    
    # train = np.hstack([X_train, y_train])
    train = X_train
    
    for i in range(qtd_modelos):
        
        print('Training model: ', i)
        tam = len(train)
       
        indices = reamostragem(train, tam)
        # indices = np.arange(tam)
        
        
        particao_x = X_train[indices]
        particao_y = Y_train[indices]

        
        # x_train = Xtrain[0:-tam_val, lags_acf]
        x_train = particao_x[:val_index]
        y_train = particao_y[:val_index]
        # x_val = Xtrain[-tam_val:, lags_acf]
        x_val = particao_x[val_index:]
        y_val = particao_y[val_index:]
        
        
        
        model, _ = train_svr(x_train, y_train, x_val, y_val)
        ens.append(model)
   
    return ens

In [20]:
def get_euclidian_neighbors(df, max_k, lags, val_index, test_index):
    euclidian_windows = np.zeros((df['y'].size-val_index, max_k))
    i_list = list(range(val_index, df['y'].size))             
    pool = Pool()
    results = list(tqdm(pool.imap(partial(parallel.sort_euclidian_distances, df=df, k=max_k, lags=lags, val_index=val_index, test_index=test_index), i_list), total=len(i_list)))
    for r in results:
        euclidian_windows[r[0]-val_index] = r[1]
    pool.close()
    pool.join()
    return euclidian_windows

In [21]:
def run_ensemble(execution, df, ens_size, train_proportion, val_proportion, windows, lags, max_k, plot_curves, normalize_before_metrics, differencing_order, level, p_level):
    print("\n###############################################\n")
    print("Executing: ENSEMBLE SVR")
    val_index, test_index = define_split_points(df, train_proportion, val_proportion)
    d = 0
    if differencing_order:
        df_differenced, d = difference(df, differencing_order, level, p_level)
    if(d):
        df_normalized, min_max_scaler = normalize(df_differenced, val_index, d)
    else:
        df_normalized, min_max_scaler = normalize(df, val_index, d)

    if(windows):
        df_normalized = create_windows(df_normalized, lags)
        
    x_train = df_normalized.drop(columns = ['y']).loc[:test_index-1].values
    y_train = df_normalized['y'].loc[:test_index-1].values
    ens = bagging(ens_size, x_train, y_train, val_index-lags-d)
    
    if(execution != "oracle"):
        result_index = val_index - max_k
    else:
        result_index = test_index
        
    euclidian_windows = None
    if execution == "dsla":
        euclidian_windows = get_euclidian_neighbors(df, max_k, lags, val_index, test_index)
    res = []
    for e in ens:
        y_pred, y_test = test_model(e, df_normalized, result_index, min_max_scaler)
        if d:
            y_pred, y_test = inverse_difference(d, df, df_differenced, result_index, y_pred)
        if(execution != "oracle"):
            if plot_curves:
                plot_results(y_pred[(test_index-result_index):], y_test[(test_index-result_index):])
        else:
            if plot_curves:
                plot_results(y_pred, y_test)        
        res.append(y_pred)
    return res, euclidian_windows

In [22]:
metric_res = []
params_res = []

for f in file_names:
    print("Time Series: ", f)
    df = read_file(file_prefix + f + file_suffix)

    results,  euclidian_windows= run_ensemble(execution = execution, df=df, ens_size=ens_size, train_proportion=train_proportion, val_proportion=val_proportion, plot_curves=plot_curves, max_k = max_k, **ensemble_svr)
    val_index, test_index = define_split_points(df, train_proportion, val_proportion)
    min_max_scaler = preprocessing.MinMaxScaler()
    real = df['y'].loc[test_index:].values

    if (execution == "oracle"):
        oracle_pred = oracle(results, real)            
        m = calculate_metrics(oracle_pred, real, True, min_max_scaler, df)
        metric_res.append((f, m))

    else:
        parameters = {
            'k': list(range(1,max_k +1)),
            'n': list(range(1,max_n +1)),
            'comb': ['median', 'average'],
        }
        grid = list(ParameterGrid(parameters))
        pred_results = []
        for r in results:
            temp = pd.DataFrame(data = np.zeros(df['y'].size), columns = ['y'])
            temp['y'][val_index - max_k:] = r
            pred_results.append(temp)
        best_params = parallel_gridsearch_ds(df, pred_results, val_index, test_index, parameters, execution, euclidian_windows)
        params_res.append((f, best_params))
        y_ds = test_ds(best_params, df, pred_results, val_index, test_index, execution, euclidian_windows)
        m = calculate_metrics(y_ds, real, True, min_max_scaler, df)
        metric_res.append((f, m))
        min_max_scaler.fit(df['y'].values.reshape(-1,1))
        df_ds_pred = pd.DataFrame(data = min_max_scaler.transform(y_ds.reshape(-1, 1)).flatten() , columns = [execution.upper()])
        df_ds_pred.to_csv(f'results/{execution}_pred/{f}.csv', index=False)
        
        
    

Time Series:  amz

###############################################

Executing: ENSEMBLE SVR
d:  1
Training model:  0


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:02<00:00, 29.03it/s]


Best params:  {'C': 1, 'epsilon': 0.01, 'gamma': 1, 'kernel': 'rbf'}
Best rmse:  0.09354975908387514
Training model:  1


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:02<00:00, 30.72it/s]


Best params:  {'C': 10, 'epsilon': 0.01, 'gamma': 1, 'kernel': 'rbf'}
Best rmse:  0.08885291161301666
Training model:  2


...

Best params:  {'C': 1000, 'epsilon': 0.0001, 'gamma': 0.01, 'kernel': 'rbf'}
Best rmse:  0.05700911156312322
Training model:  98


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:01<00:00, 74.99it/s]


Best params:  {'C': 1000, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
Best rmse:  0.07686948111602708
Training model:  99


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:01<00:00, 67.95it/s]


Best params:  {'C': 1000, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
Best rmse:  0.07575858720848985


In [23]:
metrics_df = pd.DataFrame(columns=['Time Series', 'Metrics', execution.upper()])


for f in metric_res:
    f_name = f[0]
    for me, value in f[1].items():
        new_row = {'Time Series': f_name, 'Metrics': me, execution.upper(): value}
        new_row = pd.Series(new_row)
        metrics_df = pd.concat([metrics_df, new_row.to_frame().T], ignore_index=True)
        


metrics_df[execution.upper()] = metrics_df[execution.upper()].apply(lambda x: pd.to_numeric(x, errors='coerce'))
pd.set_option('display.float_format', '{:.2E}'.format)
print(metrics_df)

   Time Series Metrics   ORACLE
0          amz     MSE 2.81E-05
1          amz    MAPE 4.26E+00
2          amz     ARV 8.01E-03
3          amz     MAE 2.53E-03
4          amz    RMSE 5.30E-03
..         ...     ...      ...
65        wine     ARV 2.55E-03
66        wine     MAE 1.37E-03
67        wine    RMSE 3.24E-03
68        wine   NRMSE 9.93E-03
69        wine   SMAPE 5.63E+00

[70 rows x 3 columns]


In [24]:
if execution != "oracle":
    params_df = pd.DataFrame(columns=['Time Series', 'k', 'n', 'comb'])

    for f in params_res:
        new_row = {'Time Series': f[0]}
        for key, value in f[1].items():
            new_row[key] = value
        new_row = pd.Series(new_row)
        params_df = pd.concat([params_df, new_row.to_frame().T], ignore_index=True)


    params_df

In [25]:
if execution != "oracle":
    params_df.to_csv(f'results/params_results_{execution}_bagging.csv', index=False)
metrics_df.to_csv(f'results/metrics_results_{execution}_bagging.csv', index=False)